In [20]:
import os
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from collections import defaultdict

In [21]:
# annotation = {train.json dataset file 경로}
# annotation = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'
# annotation = '/opt/ml/input/data/ICDAR17_all/ufo/all_2017.json'
annotation = '/opt/ml/input/data/Aihub_Korean/ufo/aihub.json'

with open(annotation) as f: 
    data = json.load(f)

In [22]:
file_list = os.listdir('/opt/ml/input/data/Aihub_Korean/images')

In [23]:
file_list = [filelist for filelist in file_list if os.path.splitext(filelist)[1] == ".jpg" and os.path.splitext(filelist)[0][0] != '.']

In [5]:
# for i in file_list:
#     if i[0] != 'i':
#         print(i)

In [18]:
file_list

['img_3569.jpg',
 'img_1903.jpg',
 'img_5901.jpg',
 'img_5875.jpg',
 'img_3777.jpg',
 'img_4282.jpg',
 'img_1575.jpg',
 'img_4563.jpg',
 'img_3897.jpg',
 'img_1876.jpg',
 'img_5804.jpg',
 'img_363.jpg',
 'img_1511.jpg',
 'img_314.jpg',
 'img_3648.jpg',
 'img_4508.jpg',
 'img_959.jpg',
 'img_5991.jpg',
 'img_6142.jpg',
 'img_3842.jpg',
 'img_2162.jpg',
 'img_1128.jpg',
 'img_5872.jpg',
 'img_3392.jpg',
 'img_3787.jpg',
 'img_2175.jpg',
 'img_2237.jpg',
 'img_1665.jpg',
 'img_2109.jpg',
 'img_6155.jpg',
 'img_4601.jpg',
 'img_587.jpg',
 'img_3778.jpg',
 'img_2367.jpg',
 'img_3609.jpg',
 'img_863.jpg',
 'img_4579.jpg',
 'img_4499.jpg',
 'img_1424.jpg',
 'img_6320.jpg',
 'img_1183.jpg',
 'img_4506.jpg',
 'img_1578.jpg',
 'img_3291.jpg',
 'img_4659.jpg',
 'img_3853.jpg',
 'img_2284.jpg',
 'img_4549.jpg',
 'img_3627.jpg',
 'img_2227.jpg',
 'img_2385.jpg',
 'img_6117.jpg',
 'img_4614.jpg',
 'img_550.jpg',
 'img_5814.jpg',
 'img_945.jpg',
 'img_3304.jpg',
 'img_418.jpg',
 'img_5635.jpg',
 'img

In [24]:
len(file_list)

1000

In [25]:
get_id = {"ko":0, "en":1}
get_category = ["ko", "en"]

In [12]:
# var = []
# for idx, file in enumerate(file_list):
#     for i in data['images'][file]['words']:
#         var.append((idx, get_id[data["images"][file]["words"][i]['language']]))

KeyError: 'F96E9B49C5EB9EA93FFF5009FAD70AD2.jpg'

In [26]:
var = [(idx, get_id[data["images"][file]["words"][i]['language']]) for idx, file in enumerate(file_list) for i in data["images"][file]["words"]]

X = np.ones((len(var),1))                   # 각 ananotation 의 index
y = np.array([v[1] for v in var])           # cateogry_id 모음
groups = np.array([v[0] for v in var])      # image_id 모음


cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=41)

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx][:10]) # image_id
    print(" ", y[train_idx][:10])           # category_id
    print(" TEST:", groups[val_idx][:10])
    print(" ", y[val_idx][:10])

TRAIN: [0 0 0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
 TEST: [1 1 1 1 1 1 1 1 1 1]
  [0 0 0 0 0 1 0 1 0 0]
TRAIN: [0 0 0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
 TEST: [2 2 2 2 2 2 4 4 4 4]
  [1 1 1 1 1 1 0 0 0 0]
TRAIN: [0 0 0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
 TEST: [6 6 6 6 6 6 6 6 6 6]
  [1 1 1 0 0 0 0 0 1 1]
TRAIN: [1 1 1 1 1 1 1 1 1 1]
  [0 0 0 0 0 1 0 1 0 0]
 TEST: [0 0 0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
TRAIN: [0 0 0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
 TEST: [8 8 8 8 8 8 8 8 8 8]
  [0 0 0 0 0 0 0 0 0 0]


In [27]:
from collections import Counter
import pandas as pd

In [28]:
def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

In [29]:
distrs = [get_distribution(y)]
index = ['training set']

In [30]:
for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0
    
    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')

In [31]:
pd.DataFrame(distrs, index=index, columns = [get_category[i] for i in range(np.max(y) + 1)])

,ko,en
training set,63.74%,36.26%
train - fold0,65.45%,34.55%
val - fold0,56.47%,43.53%
train - fold1,63.28%,36.72%
val - fold1,65.50%,34.50%
train - fold2,63.94%,36.06%
val - fold2,62.95%,37.05%
train - fold3,62.88%,37.12%
val - fold3,67.00%,33.00%
train - fold4,63.10%,36.90%


In [32]:
# annotation = {dataset 경로/K-fold}
output_filename = "/opt/ml/input/data/Aihub_Korean/ufo/K-fold"

In [33]:
for idx, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    train_images, val_images = dict(), dict()
    train_annotations, val_annotations = [], []
    for i in groups[train_idx]: # image_id
        train_images[file_list[i]] = data["images"][file_list[i]].copy()
    for i in groups[val_idx]:   # image_id
        val_images[file_list[i]] = data["images"][file_list[i]].copy()

    train_split = {
            "images": train_images
        }

    val_split = {
            "images": val_images
        }
    
    output_files = []
    for split_type, split in zip(["train", "val"], [train_split, val_split]):
        output_files.append(output_filename + f"_{split_type}{idx+1}.json")
        with open(output_files[-1], "w") as f:
            json.dump(split, f, indent=2)

print("Split Done !")


Split Done !
